In [5]:
import os
import pandas as pd
import geocoder
import googlemaps
from datetime import datetime
import time
import tqdm
import warnings
warnings.filterwarnings('ignore')

from dotenv import load_dotenv
load_dotenv(dotenv_path='../../../../../.env')
api_key = os.getenv('GEOCODER_API_KEY')
os.environ['GEOCODER_API_KEY'] = api_key

# load csv

In [2]:
path = os.path.abspath('./vaccination_location/')

In [3]:
data = pd.DataFrame()

for file in os.listdir(path):
    if '.csv' in file:
        print(file)
        try:
            temp = pd.read_csv(path + '/' + file, encoding='big5')
        except:
            temp = pd.read_csv(path + '/' + file, encoding='utf8')
        data = pd.concat([data, temp])

HsinchuCounty.csv
ChanghuaCounty.csv
PenghuCounty.csv
MiaoliCounty.csv
KaohsiungCity.csv
vaccinaiton_address.csv
TaitungCounty.csv
HualienCounty.csv
TaipeiCity.csv
TaichungCity.csv
NantouCounty.csv


In [13]:
data = data.reset_index(drop=True)
data

,admin,address
0,新竹縣,新竹縣新埔鎮福德街535號
1,新竹縣,新竹縣湖口鄉達生路506號
2,新竹縣,新竹縣峨眉鄉峨眉村峨眉街2鄰5號
3,新竹縣,新竹縣竹東鎮中正路221號
4,新竹縣,新竹縣新豐鄉重興村萃豐路21號
...,...,...
1524,南投縣,集集鎮民生路106號
1525,南投縣,竹山鎮雲林里公所路126號
1526,南投縣,草屯鎮草溪路882巷7號
1527,南投縣,埔里鎮中山路二段256號


# transform addresses from files

In [25]:
new_data = pd.DataFrame(columns = ['admin', 'address', 'longitude', 'latitude'])

In [27]:
not_found = set()

In [29]:
gmaps = googlemaps.Client(key=api_key)

cnt = 0
for rows in data.itertuples():
    if rows.address not in set(new_data.address):
        print(cnt, rows.address)
        geocode_result = gmaps.geocode(rows.address)
        try:
            lat = geocode_result[0]['geometry']['location']['lat']
            lng = geocode_result[0]['geometry']['location']['lng']
        except:
            print(f'Not found: {rows.address}')
            not_found.add(rows.address)
            lat = None
            lng = None
        
        new_data = new_data._append({
            'admin':rows.admin,
            'address':rows.address,
            'latitude':lat,
            'longitude':lng
        }, ignore_index=True)
        
        cnt += 1
        if cnt % 50 == 0:
            new_data.to_csv('./vaccination_coor.csv', index=False)
            print('=============================================')
            time.sleep(5)
            
new_data.to_csv('./vaccination_coor.csv', index=False)
print('done')

0 新竹縣新埔鎮福德街535號
1 新竹縣湖口鄉達生路506號
2 新竹縣峨眉鄉峨眉村峨眉街2鄰5號
3 新竹縣竹東鎮中正路221號
4 新竹縣新豐鄉重興村萃豐路21號
5 新竹縣關西鎮中山路207號
6 新竹縣芎林鄉文山路532號
7 新竹縣寶山鄉雙溪村雙園路二段273號
8 新竹縣橫山鄉大肚村新興街141巷10號
9 新竹縣北埔鄉北埔村北埔街90號
10 新竹縣尖石鄉嘉樂村二鄰61號
11 新竹縣五峰鄉大隘村6鄰99號
12 新竹縣竹北市光明二街89號
13 新竹縣竹北市興隆路一段199號
14 新竹縣竹北市生醫路一段2號(竹北院區)
15 新竹縣竹北市博愛街331號
16 新竹縣竹東鎮仁愛路196號
17 新竹縣湖口鄉忠孝路29號
18 新竹縣竹北市縣政二路69號
19 新竹縣竹北市仁和街7號2-4樓
20 新竹縣竹北市光明一路242號
21 新竹縣新埔鎮中正路470巷3號
22 新竹縣竹北市嘉豐六路一段31號
23 新竹縣竹北市勝利三街38號
24 新竹縣新豐鄉建興路二段753號
25 新竹縣湖口鄉安宅9街81號
26 新竹縣新豐鄉新市路3號
27 新竹縣竹北市勝利五路27號
28 新竹縣竹北市中山路460號
29 新竹縣竹北市光明一路392-1號1樓
30 新竹縣竹北市光明六路303-2號
31 新竹縣竹北市仁和街七號一樓
32 新竹縣新埔鎮中正路330號
33 彰化市民族路270號
34 彰化市曉陽路200號
35 彰化市三民路207號
36 彰化市華山路142號
37 彰化市中正路一段258號
38 彰化市中正路415號
39 彰化市東民街11號
Not found: 彰化市東民街11號
40 彰化市中山路二段250號
41 彰化市中正路一段432號
42 彰化市辭修路82號
43 彰化市崙平南路333號
44 彰化市中正路2段426號
45 彰化巿永安街446號
46 彰化縣員林市員水路二段482號
47 彰化縣員林市員東路一段55號
48 員林市中山南路72號
49 彰化縣員林市正興街370號
50 彰化縣員林市三民街72號
51 彰化縣員林市三民街38-1號
52 彰化縣員林市育英路37-12號
53 彰化縣員林市中山路2段114號
54 彰化縣員林市大同路一段285巷15號
55 彰化縣員林市林森路308號
56 彰化縣員林市正興街368號2樓

418 新北市土城區中央路2段238號
419 新北市土城區明德路1段2號1樓
420 新北市土城區裕民路144號1樓
421 新北市土城區中央路2段223之9號1樓
422 新北市土城區清水路233號1樓
423 新北市土城區學府路一段188號
424 新北市土城區延和路112號1樓
425 新北市土城區裕民路83號1、2樓
426 新北市土城區中央路四段286號
427 新北市土城區中央路二段312號1樓
428 新北市土城區裕民路131號
429 新北市土城區金城路3段137號1樓
430 新北市土城區立雲街3號1樓
431 新北市土城區青仁路29號1樓
432 新北市土城區青雲路184號1樓
433 新北市土城區中央路2段219號
434 新北市土城區光明街58號1樓及夾層
435 新北市土城區延和路114號1樓
436 新北市土城區裕民路27號
437 新北市中和區南山路4巷3號
438 新北市中和區中正路291號
439 新北市中和區連城路49號
440 新北市中和區自強路58號
441 新北市中和區華新街49號
442 新北市中和區宜安路77號
443 新北市中和區大勇街23號1樓
444 新北市中和區南華路22號
445 新北市中和區宜安路134－1號
446 新北市中和區興南路一段91號1樓
447 新北市中和區華新街176號
448 新北市中和區中正路778-1.780-1號1樓
449 新北市中和區景新街389號
450 新北市五股區新城三路11號
451 新北市五股區凌雲路一段73號
Not found: 新北市五股區凌雲路一段73號
452 新北市五股區成泰路2段61號
453 新北市五股區自強路36號
454 新北市平溪區公園街17號
455 新北市永和區秀朗路一段137號
456 新北市永和區中興街80號
457 新北市永和區中正路436號
458 新北市永和區得和路198號
459 新北市永和區得和路245號
460 新北市永和區文化路78號
461 新北市永和竹林路81號
462 新北市永和區竹林路13號
463 新北市永和區成功路二段133號1樓
464 新北市永和區仁愛路147號1樓
465 新北市永和區得和路275號
466 新北市永和區中正路106號
467 新北市永和區福和路250號
468 新北市永和區保安路77號1樓


834 臺南市下營區下營里中正北路18號1樓
835 臺南市下營區營前里中正南路55號1樓
836 臺南市下營區營前里中山路二段137號
837 臺南市六甲區二甲里中山路101．103號
838 臺南市六甲區六甲里建國街74號
839 臺南市六甲區中正路178號
840 臺南市六甲區六甲里中正路531號
841 臺南市六甲區甲南里珊瑚路26號
842 臺南市六甲區復興街55號
843 臺南市六甲區二甲里26鄰中華路110號
844 臺南市官田區隆田里中山路一段128號
845 臺南市大內區大內里2號
846 台南市佳里區新生路272號
847 臺南市佳里區忠仁里中山路188號1樓
848 臺南市佳里區東寧里文化路205號
849 臺南市佳里區安西里佳北路1號2－4樓
850 臺南市佳里區鎮山里延平路72號2樓
851 臺南市佳里區佳興里佳里興606號
852 臺南市佳里區東寧里進學路159號
853 臺南市學甲區仁得里華宗路365號
854 臺南市學甲區仁得里建國路38號1樓
855 臺南市西港區慶安路95之8號
856 臺南市西港區西港里中山路350號
857 臺南市西港區慶安里中山路325號
858 臺南市西港區西港里中山路368號
859 臺南市七股區大埕里379號
860 臺南市將軍區忠興里185－6號
861 臺南市北門區北門里港北50號
862 臺南市新化區民權街15號
863 臺南市新化區太平里忠孝路126－1號1樓
864 臺南市新化區忠孝路285號
865 臺南市新化區武安里和平街171號
866 台南市新化區忠孝路120號
867 臺南市新化區中山路193號1樓
868 臺南市新化區清水里中山路489號1樓及491之1號
869 臺南市新化區健康路十三鄰143號
870 臺南市善化區文昌里中正路387號1樓
871 臺南市善化區大成路312－1號
872 臺南市善化區光文里中山路204號
873 臺南市善化區光文里中山路296號
874 臺南市善化區光文里中山路200號
875 臺南市新市區南科三路7號一樓
876 臺南市新市區新和里民權路25號
877 臺南市新市區大營里大營130號之10
878 臺南市新市區中正路331－1號
879 臺南市新市區新市里和平街31號
880 臺南市新市區新和里民生路46號
881 臺南市新市區新市里中興街12

1191 613台灣嘉義縣朴子市平和路11之6號
1192 614台灣嘉義縣東石鄉3-7號
1193 608台灣嘉義縣水上鄉正義路145號
1194 608台灣嘉義縣水上鄉正義路53號
1195 624台灣嘉義縣義竹鄉仁里村422-1號422-1
1196 611台灣嘉義縣鹿草鄉長壽路226巷2號
1197  基隆市信義區信二路268號
1198  基隆市中正區義二路2巷6號
1199  基隆市中正區中正路30號
1200  基隆市中正區義一路41號1、2、3、5、6、7樓
1201  基隆市中正區義一路76號3、4樓
1202  基隆市中正區北寧路319號
1203  基隆市中正區義一路62號1樓
1204  基隆市七堵區南興路101號
1205  基隆市七堵區開元路83號
1206  基隆市七堵區開元路59－1號、地下室1樓、1樓
1207  基隆市七堵區百三街29號
1208  基隆市仁愛區忠二路3－14號
1209  基隆市仁愛區仁二路二五一號一樓
1210  基隆市仁愛區孝三路80號
1211  基隆市仁愛區忠一路20號1-3樓及23號1樓
1212  基隆市仁愛區劉銘傳路1-5號6樓
1213  基隆市中山區安一路3號
1214  基隆市七堵區福二街320號
1215  基隆市中正區義一路46號
1216  基隆市中正區新豐街450號1、2樓
1217  連江縣南竿鄉復興村217號
1218  連江縣北竿鄉塘岐村281－6號
1219  連江縣莒光鄉青帆村14－1號
1220  連江縣莒光鄉大坪村30－1號
1221  連江縣東引鄉中柳村123號
1222 臺東縣臺東市長沙街303巷1號
1223 台東縣台東市杭州街2號
1224 臺東縣臺東市開封街350號
1225 臺東縣臺東市更生路1000號
1226 臺東縣臺東市五權街1號
1227 臺東縣臺東市光明路124號
1228 臺東縣臺東市中華路一段396號
1229 臺東縣臺東市中正路295號
1230 臺東縣臺東市中華路一段317號
1231 臺東縣臺東市中正路334號
1232 臺東縣臺東市四維路三段36號
1233 臺東縣臺東市鄭州街3號
1234 臺東縣臺東市更生路199號
1235 臺東縣臺東市中山路224號
1236 臺東縣臺東市中興路四段495號
1237 臺東縣臺東市大同路51號
12

# add new data

In [30]:
data2 = pd.read_csv(path + '/' +'vaccinaiton_address.csv')

In [44]:
new_data = pd.read_csv('./vaccination_coor.csv', encoding='utf-8')

In [45]:
new_data_address = set(new_data.address)

In [46]:
len(new_data)

1508

In [47]:
len(set(new_data.admin))

22

In [48]:
set(new_data.admin)

{'南投縣',
 '台中市',
 '台北市',
 '台南市',
 '台東縣',
 '嘉義市',
 '嘉義縣',
 '基隆市',
 '宜蘭縣',
 '屏東縣',
 '彰化市',
 '新北市',
 '新竹市',
 '新竹縣',
 '桃園市',
 '澎湖縣',
 '花蓮縣',
 '苗栗縣',
 '連江縣',
 '金門縣',
 '雲林縣',
 '高雄市'}

In [37]:
gmaps = googlemaps.Client(key=api_key)

cnt = 0
for rows in data2.itertuples():
    if rows.address not in set(new_data.address):
        print(cnt, rows.address)
        geocode_result = gmaps.geocode(rows.address)
        try:
            lat = geocode_result[0]['geometry']['location']['lat']
            lng = geocode_result[0]['geometry']['location']['lng']
        except:
            print(f'Not found: {rows.address}')
            not_found.add(rows.address)
            lat = None
            lng = None
        
        new_data = new_data._append({
            'admin':rows.admin,
            'address':rows.address,
            'latitude':lat,
            'longitude':lng
        }, ignore_index=True)
        
        cnt += 1
        if cnt % 50 == 0:
            new_data.to_csv('./vaccination_coor.csv', index=False)
            print('=============================================')
            time.sleep(5)

new_data.to_csv('./vaccination_coor.csv', index=False)

In [35]:
new_data

,admin,address,latitude,longitude,lat,lng
0,新竹縣,新竹縣新埔鎮福德街535號,24.826151,121.072616,NaN,NaN
1,新竹縣,新竹縣湖口鄉達生路506號,24.896766,121.047253,NaN,NaN
2,新竹縣,新竹縣峨眉鄉峨眉村峨眉街2鄰5號,24.688743,121.021538,NaN,NaN
3,新竹縣,新竹縣竹東鎮中正路221號,24.735935,121.092825,NaN,NaN
4,新竹縣,新竹縣新豐鄉重興村萃豐路21號,24.893878,120.997893,NaN,NaN
...,...,...,...,...,...,...
0,連江縣,連江縣南竿鄉復興村217號,NaN,NaN,26.164559,119.958154
1,連江縣,連江縣北竿鄉塘岐村281－6號,NaN,NaN,26.224482,119.994287
2,連江縣,連江縣莒光鄉青帆村14－1號,NaN,NaN,25.971849,119.935024
3,連江縣,連江縣莒光鄉大坪村30－1號,NaN,NaN,25.960647,119.972608


# city_mapping

In [49]:
city_mapping = {
    '嘉義縣': 'Chiayi County',
    '桃園市': 'Taoyuan',
    '苗栗縣': 'Miaoli',
    '高雄市': 'Kaohsiung',
    '南投縣': 'Nantou',
    '雲林縣': 'Yunlin',
    '新竹縣': 'Hsinchu County',
    '彰化縣': 'Changhua',
    '宜蘭縣': 'Yilan',
    '台北市': 'Taipei',
    '台東縣': 'Taitung',
    '屏東縣': 'Pingtung',
    '澎湖縣': 'Penghu',
    '新北市': 'New Taipei',
    '連江縣': 'Lienchiang',
    '基隆市': 'Keelung',
    '金門縣': 'Kinmen',
    '新竹市': 'Hsinchu City',
    '台南市': 'Tainan',
    '台中市': 'Taichung',
    '嘉義市': 'Chiayi City',
    '花蓮縣': 'Hualien'
}

new_data['admin'] = new_data['admin'].map(city_mapping)

In [50]:
len(set(new_data['admin']))

22

In [51]:
new_data.to_csv('./vaccination_coor.csv', index=False)

In [52]:
new_data

,admin,address,longitude,latitude
0,Hsinchu County,新竹縣新埔鎮福德街535號,121.072616,24.826151
1,Hsinchu County,新竹縣湖口鄉達生路506號,121.047253,24.896766
2,Hsinchu County,新竹縣峨眉鄉峨眉村峨眉街2鄰5號,121.021538,24.688743
3,Hsinchu County,新竹縣竹東鎮中正路221號,121.092825,24.735935
4,Hsinchu County,新竹縣新豐鄉重興村萃豐路21號,120.997893,24.893878
...,...,...,...,...
1503,Nantou,集集鎮民生路106號,120.786601,23.828949
1504,Nantou,竹山鎮雲林里公所路126號,120.672477,23.759486
1505,Nantou,草屯鎮草溪路882巷7號,120.680031,23.973502
1506,Nantou,埔里鎮中山路二段256號,120.968586,23.967112


# deal with not-found addresses

In [53]:
not_found

{'彰化市東民街11號', '新北市五股區凌雲路一段73號', '金門縣民生路9-7號'}

In [57]:
new_data[new_data['latitude'].isna()==True]

,admin,address,longitude,latitude
451,New Taipei,新北市五股區凌雲路一段73號,NaN,NaN
684,Kinmen,金門縣民生路9-7號,NaN,NaN


In [56]:
new_data.loc[39, 'longitude'] = 120.5392751
new_data.loc[39, 'latitude'] = 24.0721048

In [58]:
new_data.loc[451, 'longitude'] = 121.4510446
new_data.loc[451, 'latitude'] = 25.1010949

In [59]:
new_data.loc[684, 'longitude'] = 118.3199068
new_data.loc[684, 'latitude'] = 24.4335238

In [61]:
new_data

,admin,address,longitude,latitude
0,Hsinchu County,新竹縣新埔鎮福德街535號,121.072616,24.826151
1,Hsinchu County,新竹縣湖口鄉達生路506號,121.047253,24.896766
2,Hsinchu County,新竹縣峨眉鄉峨眉村峨眉街2鄰5號,121.021538,24.688743
3,Hsinchu County,新竹縣竹東鎮中正路221號,121.092825,24.735935
4,Hsinchu County,新竹縣新豐鄉重興村萃豐路21號,120.997893,24.893878
...,...,...,...,...
1503,Nantou,集集鎮民生路106號,120.786601,23.828949
1504,Nantou,竹山鎮雲林里公所路126號,120.672477,23.759486
1505,Nantou,草屯鎮草溪路882巷7號,120.680031,23.973502
1506,Nantou,埔里鎮中山路二段256號,120.968586,23.967112


In [62]:
new_data.to_csv('./vaccination_coor.csv', index=False)